In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

url='https://en.wikipedia.org/wiki/2025%E2%80%9326_Premier_League'

headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
}

page=requests.get(url,headers=headers)

soup=BeautifulSoup(page.text,'lxml')

table=soup.find_all('table',class_='wikitable')[3]



head=table.find_all('th',scope='col')

head_table=[title.text for title in head]
columns=[head.strip() for head in head_table]


df=pd.DataFrame(columns=columns)
df


,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation


In [2]:
rows=table.find_all('tr')

for row in rows[1:]:
    row_header=row.find('th',scope='row')
    team=row_header.text.strip() if row_header else ""

    cells=[td.text.strip() for td in row.find_all('td')]

    row_data=[team]+cells

    if len(row_data)<len(columns):
        row_data+=['']*(len(columns)-len(row_data))

    df.loc[len(df)]=row_data


df.head()






,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts,Qualification or relegation
0,Arsenal,1,17,12,3,2,31,10,+21,39,Qualification for the Champions League league ...
1,Manchester City,2,17,12,1,4,41,16,+25,37,
2,Aston Villa,3,17,11,3,3,27,18,+9,36,
3,Chelsea,4,17,8,5,4,29,17,+12,29,
4,Liverpool,5,17,9,2,6,28,25,+3,29,Qualification for the Europa League league pha...


## dropped the qualification or relegation columns since we do not need it to make predictions


In [3]:
df.drop(columns='Qualification or relegation',inplace=True)
df

,Pos,Team,Pld,W,D,L,GF,GA,GD,Pts
0,Arsenal,1,17,12,3,2,31,10,+21,39
1,Manchester City,2,17,12,1,4,41,16,+25,37
2,Aston Villa,3,17,11,3,3,27,18,+9,36
3,Chelsea,4,17,8,5,4,29,17,+12,29
4,Liverpool,5,17,9,2,6,28,25,+3,29
5,Sunderland,6,17,7,6,4,19,17,+2,27
6,Manchester United,7,17,7,5,5,31,28,+3,26
7,Crystal Palace,8,17,7,5,5,21,19,+2,26
8,Brighton & Hove Albion,9,17,6,6,5,25,23,+2,24
9,Everton,10,17,7,3,7,18,20,−2,24


In [4]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense



df.rename(columns={'Team':'Pos',"Pos":"Team"},inplace=True)
df






c:\Users\Administrator\PycharmProjects\quiz game\.venv\Lib\site-packages\keras\src\export\tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


,Team,Pos,Pld,W,D,L,GF,GA,GD,Pts
0,Arsenal,1,17,12,3,2,31,10,+21,39
1,Manchester City,2,17,12,1,4,41,16,+25,37
2,Aston Villa,3,17,11,3,3,27,18,+9,36
3,Chelsea,4,17,8,5,4,29,17,+12,29
4,Liverpool,5,17,9,2,6,28,25,+3,29
5,Sunderland,6,17,7,6,4,19,17,+2,27
6,Manchester United,7,17,7,5,5,31,28,+3,26
7,Crystal Palace,8,17,7,5,5,21,19,+2,26
8,Brighton & Hove Albion,9,17,6,6,5,25,23,+2,24
9,Everton,10,17,7,3,7,18,20,−2,24


In [5]:
# df[['Pts','Pos','Pld','W','D','L','GA','GF']]=df[['Pts','Pos','Pld','W','D','L','GA','GF']].astype(int)
for col in ['Pts','Pos','Pld','W','D','L','GA','GF']:
    df[col]=df[col].astype(int)

df['GD']=df['GD'].astype('str')
df['GD'] = df['GD'].str.replace('−', '-', regex=False)
df['GD'] = df['GD'].str.replace('+', '', regex=False)

df['GD']=pd.to_numeric(df['GD'])

df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 20 entries, 0 to 19
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Team    20 non-null     object
 1   Pos     20 non-null     int32 
 2   Pld     20 non-null     int32 
 3   W       20 non-null     int32 
 4   D       20 non-null     int32 
 5   L       20 non-null     int32 
 6   GF      20 non-null     int32 
 7   GA      20 non-null     int32 
 8   GD      20 non-null     int64 
 9   Pts     20 non-null     int32 
dtypes: int32(8), int64(1), object(1)
memory usage: 1.1+ KB


In [6]:
df.head()

,Team,Pos,Pld,W,D,L,GF,GA,GD,Pts
0,Arsenal,1,17,12,3,2,31,10,21,39
1,Manchester City,2,17,12,1,4,41,16,25,37
2,Aston Villa,3,17,11,3,3,27,18,9,36
3,Chelsea,4,17,8,5,4,29,17,12,29
4,Liverpool,5,17,9,2,6,28,25,3,29


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


features=['Pld','W','D','L','GA','GF','GD']
target='Pts'

X=df[features].values
y=df[target].values

scaler=StandardScaler()
X_scaled=scaler.fit_transform(X)

X_train,X_test,y_train,y_test=train_test_split(X_scaled,y,test_size=0.2,random_state=42)

model=Sequential([
    Dense(128,input_dim=X_train.shape[1],activation='relu'),
    Dense(64,activation='relu'),
    Dense(32,activation='relu'),
    Dense(1)
])

model.compile(optimizer='adam',loss='mean_squared_error',metrics=['mae'])
model.summary()

history = model.fit(
    X_train, y_train,
    validation_split=0.2,
    epochs=300,        
    batch_size=4,
    verbose=1
)

c:\Users\Administrator\PycharmProjects\quiz game\.venv\Lib\site-packages\keras\src\layers\core\dense.py:106: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         1,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,393 (44.50 KB)

 Trainable params: 11,393 (44.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 8s 314ms/step - loss: 573.8599 - mae: 22.4651 - val_loss: 589.4846 - val_mae: 24.2143
Epoch 2/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 567.8929 - mae: 22.2860 - val_loss: 584.9146 - val_mae: 24.1185
Epoch 3/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 562.9606 - mae: 22.1496 - val_loss: 580.4844 - val_mae: 24.0254
Epoch 4/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - loss: 558.4464 - mae: 22.0082 - val_loss: 575.5968 - val_mae: 23.9224
Epoch 5/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 199ms/step - loss: 552.5405 - mae: 21.8576 - val_loss: 570.3259 - val_mae: 23.8110
Epoch 6/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 213ms/step - loss: 546.4527 - mae: 21.6963 - val_loss: 564.6494 - val_mae: 23.6905
Epoch 7/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 1s 209ms/step - loss: 539.3000 - mae: 21.5234 - val_loss: 558.6349 - val_mae: 23.5623
Epoch 8/300
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 87ms/step - loss: 531.4599 - mae: 21.3343 - val_loss: 551.8684 - val_mae: 23.4170
Epoch 9/300
3/3 ━━━━

In [8]:

loss, mae = model.evaluate(X_test, y_test)
print("Test Mean Absolute Error:", mae)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 526ms/step - loss: 181.9703 - mae: 11.3490
Test Mean Absolute Error: 11.349015235900879


In [9]:
loss, mae = model.evaluate(X_test, y_test)
print("Test Mean Absolute Error:", mae)



1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 212ms/step - loss: 181.9703 - mae: 11.3490
Test Mean Absolute Error: 11.349015235900879


In [10]:
y_pred = model.predict(X_test)
print("Predicted Points:", y_pred.flatten())
print("Actual Points:", y_test)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 324ms/step
Predicted Points: [49.73198   8.00389  12.758175 60.426147]
Actual Points: [39 13 19 37]


In [12]:
model.save("premier_league_points_model.h5")


import joblib
joblib.dump(scaler, "scaler.pkl")

['scaler.pkl']